# LLM Zoomcamp 2024 Competition
This competition requires the use of LLMs to solve high school mathematics problems. The task is to develop models that can accurately solve the problems and then the predictions submitted.

In [13]:
# Import required modules.
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
import re
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from scorer import score

In [3]:
# Set up OpenAI API key.
load_dotenv()

True

In [4]:
# Application Design.
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0, max_tokens=1500)

In [5]:
def get_answer(question):
    prompt = f"""Please solve the following high school math problem step by step. Explain your reasoning clearly and provide the final answer.

{question}

Step-by-step solution and final answer:"""

    response = llm.invoke(prompt)
    return response.strip()

In [6]:
# Function to Extract Answer.
def extract_numerical_answer(text):
    # Look for patterns like "Final answer: X" or "The answer is X" at the end of the text
    match = re.search(r'(?:final answer|the answer is)[:\s]*([+-]?\d*\.?\d+)', text, re.IGNORECASE)
    if match:
        return float(match.group(1))
    else:
        # If no clear final answer, look for the last number in the text
        numbers = re.findall(r'[+-]?\d*\.?\d+', text)
        return float(numbers[-1]) if numbers else None

In [7]:
# Function to Prepare Dataset and Get Answers.
pool = ThreadPoolExecutor(max_workers=6)

def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [8]:
# create a function for processing each row.
def process_row(row):
    problem_id = row['problem_id']
    problem_text = row['problem_text']

    llm_reasoning = get_answer(problem_text)

    numerical_answer = extract_numerical_answer(llm_reasoning)

    return {
        'problem_id': problem_id,
        'problem_text': problem_text,
        'llm_reasoning': llm_reasoning,
        'answer': numerical_answer
    }

In [9]:
# Load the train data.
df_train = pd.read_csv('data/train.csv')
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [10]:
# Process row.
rows = df_train.head().to_dict(orient='records')
process_row(rows[0])

{'problem_id': 2374,
 'problem_text': 'Find the value of the expression $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$.',
 'llm_reasoning': 'To solve this problem, we will first simplify the expression $\\dfrac{17}{5} :\\dfrac{34}{3}$.\n\nTo divide fractions, we need to invert the second fraction and multiply. This can be written as:\n\n$\\dfrac{17}{5} :\\dfrac{34}{3} = \\dfrac{17}{5} \\times \\dfrac{3}{34}$\n\nNext, we can simplify the fractions by canceling out any common factors. In this case, we can divide both the numerator and denominator of the first fraction by 17, and the second fraction by 2. This gives us:\n\n$\\dfrac{1}{5} \\times \\dfrac{3}{17} = \\dfrac{3}{85}$\n\nNow, we can substitute this simplified fraction back into the original expression:\n\n$\\dfrac{17}{5} :\\dfrac{34}{3} +1.3 = \\dfrac{3}{85} + 1.3$\n\nTo add fractions, we need to have a common denominator. In this case, we can multiply the first fraction by $\\dfrac{17}{17}$ to get a common denominator of 85. This gives 

In [11]:
# Get df_results.
results = map_progress(pool, rows, process_row)
df_results = pd.DataFrame(results)
df_results

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.15s/it]


,problem_id,problem_text,llm_reasoning,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,"To solve this problem, we will first simplify ...",17.0
1,4723,"In a company of 30 people, 25 use the social n...","1) In this company, there will be 10 people wh...",24.0
2,7135,The number of road traffic accidents (RTAs) in...,Step 1: Let x be the number of RTAs in the win...,32.0
3,5814,Find the value of the expression $\dfrac{2\str...,"To solve this problem, we will use the propert...",256.0
4,9237,A traveler from Moscow wants to visit four cit...,To visit all four cities and spend less than 5...,46.0


100%|█████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:03<00:00,  1.19s/it]

In [12]:
# Show the head of train.
df_train.head()

,problem_id,problem_text,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,1.6
1,4723,"In a company of 30 people, 25 use the social n...",24
2,7135,The number of road traffic accidents (RTAs) in...,32
3,5814,Find the value of the expression $\dfrac{2\str...,256
4,9237,A traveler from Moscow wants to visit four cit...,53


In [14]:
# Score the run.
score(df_train.head(), df_results, 'problem_id')

0.6

In [15]:
# Put all together in a function.
def prepare_prompts_and_get_answers(df):
    rows = df.to_dict(orient='records')
    results = map_progress(pool, rows, process_row)
    return pd.DataFrame(results)

In [16]:
# Return the results head.
df_train_results = prepare_prompts_and_get_answers(df_train)
df_train_results.head()

100%|█████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:03<00:00,  1.57it/s]


,problem_id,problem_text,llm_reasoning,answer
0,2374,Find the value of the expression $\dfrac{17}{5...,"To solve this problem, we will first simplify ...",17.0
1,4723,"In a company of 30 people, 25 use the social n...","1) In this company, there will be 10 people wh...",24.0
2,7135,The number of road traffic accidents (RTAs) in...,Step 1: Let x be the number of RTAs in the win...,32.0
3,5814,Find the value of the expression $\dfrac{2\str...,"To solve this problem, we will use the propert...",256.0
4,9237,A traveler from Moscow wants to visit four cit...,To visit all four cities and spend less than 5...,46.0


 97%|███████████████████████████████████████████████████████████████████████████████████████▎  | 97/100 [00:59<00:02,  1.42it/s]

In [17]:
# Get the total score.
score(df_train, df_train_results, 'problem_id')

0.39

In [18]:
# Prepare the test submission.
df_test = pd.read_csv('data/test.csv')

df_test_results = prepare_prompts_and_get_answers(df_test)

submission = df_test_results[['problem_id', 'answer']]
submission.to_csv('starter_notebook_submission.csv', index=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:03<00:00,  1.58it/s]
